## Class Object
Java reflection uses instances of `Class` to represent types. Some important methods of the class:
```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
String getName()            Returns the fully qualified name of the target Class object
Class getComponentType()    If the target object is a Class object for an array, returns the
                            Class object representing the component type
boolean isArray()           Returns true if and only if the target Class object represents
                            an array
boolean isInterface()       Returns true if and only if the target Class object represents
                            an interface
boolean isPrimitive()       Returns true if and only if the target Class object represents
                            a primitive type or void
------------------------------------------------------------------------------------------------
```

In [1]:
Class primitiveClass = int.class;
System.out.println("Class " + primitiveClass.getName() + " is primitive? " + primitiveClass.isPrimitive());

Class arrayClass = String[].class;
System.out.println("Class " + arrayClass.getName() + " is array? " + arrayClass.isArray());

Class int is primitive? true
Class [Ljava.lang.String; is array? true


Primitive, array, and interface types are limited in the sense that we can't create instance object from those.  

**Introspectig Inheritance Hierarchy:** methods of `Class` for dealing with inheritance and interface implementation:
```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Class[] getInterfaces()          Returns an array of Class objects that represent the direct
                                 superinterfaces of the target Class object
Class getSuperclass()            Returns the Class object representing the direct superclass
                                 of the target Class object or null if the target represents
                                 Object, an interface, a primitive type, or void
boolean isAssignableFrom(Class cls) 
                                 Returns true if and only if the class or interface represented
                                 by the target Class object is either the same as or a superclass
                                 of or a superinterface of the specified Class parameter
boolean isInstance(Object obj)   Returns true if and only if the specified Object is assignment-
                                 compatible with the object represented by the target
                                 Class object
------------------------------------------------------------------------------------------------
```

## Method Object
Represents method of a class. To get a method for a class:

In [ ]:
Class vectorClass = Vector.class;
try {
    Method copyInto = vectorClass.getMethod("copyInto", Object[].class);
    Method trimToSize = vectorClass.getMethod("trimToSize");
    Method get = vectorClass.getMethod("get", int.class);
    Method set = vectorClass.getMethod("set", int.class, Object.class);
} catch (NoSuchMethodException e) {
    throw new RuntimeException(e);
}

`getMethod` returns matching public method defined in the class or its superclasses or superinterfaces. Once we have reference to the method, we can invoke it (among other operations listed):

```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Class getDeclaringClass()       Returns the Class object that declared the method represented
                                by this Method object
Class[] getExceptionTypes()     Returns an array of Class objects representing the types of
                                the exceptions declared to be thrown by the method represented
                                by this Method object
int getModifiers()              Returns the modifiers for the method represented by this
                                Method object encoded as an int
String getName()                Returns the name of the method represented by this Method
                                object
Class[] getParameterTypes()     Returns an array of Class objects representing the formal
                                parameters in the order in which they were declared
Class getReturnType()           Returns the Class object representing the type returned by
                                the method represented by this Method object
Object invoke(Object obj, Object[] args) 
                                Invokes the method represented by this Method object on
                                the specified object with the arguments specified in the
                                Object array
------------------------------------------------------------------------------------------------
```

**Dynamic Invocation:** is the ability of program to call a method on an object at runtime without specifying which method at compile time. At compile time, we may not know the exact type of the object on which the method is being invoked, therefore we do not know the exact method that will be called.

In [ ]:
try {
    trimToSize.invoke(vector);
    get.invoke(vector, 0);
    set.invoke(vector, 0, 4);
} catch (InvocationTargetException e) { //  wraps exception thrown by an invoked method or constructor.
    throw new RuntimeException(e);
} catch (IllegalAccessException e) { // class calling invoke does not have appropriate access privileges for the method
    throw new RuntimeException(e);
}

In addition to `getMethod`, we also have the following methods available on `Class` to query for `Method`s:
```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Method getMethod (String name,Class[] parameterTypes)
                                        Returns a Method object that represents a public
                                        method (either declared or inherited) of the target
                                        Class object with the signature specified by the
                                        second parameters
Method[] getMethods ()                  Returns an array of Method objects that represent
                                        all of the public methods (either declared or inherited)
                                        supported by the target Class object
Method getDeclaredMethod (String name, Class[] parameterTypes)
                                        Returns a Method object that represents a declared method
                                        of the target Class object with the signature specified
                                        by the second parameters (doesn't look into parent class)
Method[] getDeclaredMethods ()          Returns an array of Method objects that represent
                                        all of the methods declared by the target Class object
------------------------------------------------------------------------------------------------
```

## Field Object
The following `Class` methods help us obtain reference to `Field` objects:
```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Field getField(String name)          Returns a Field object that represents the specified public
                                     member field of the class or interface represented by this
                                     Class object
Field[] getFields()                  Returns an array of Field objects that represents all the
                                     accessible public fields of the class or interface 
                                     represented by this Class object
Field getDeclaredField( String name ) 
                                     Returns a Field object that represents the specified
                                     declared field of the class or interface represented by this
                                     Class object
Field[] getDeclaredFields()          Returns an array of Field objects that represents each
                                     field declared by the class or interface represented by this
                                     Class object
------------------------------------------------------------------------------------------------
```

As an example, to obtain all fields of an object:

In [ ]:
public Field[] getAllFields(Object obj) {
    Class clazz = obj.getClass();
    List<Field> fieldList = new ArrayList<>();
    
    while(clazz != null) {
        Field[] fields = clazz.getDeclaredFields();
        for(Field field: fields){
            fieldList.add(field);
        }
        clazz = clazz.getSuperclass();
    }
    
    return (Field[]) fieldList.toArray(new Field[fieldList.size()]);
}

Some useful methods provided by `Field`:

```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Class getType()                     Returns the Class object that represents the declared
                                    type for the field represented by this Field object
Class getDeclaringClass()           Returns the Class object that declared the field
                                    represented by this Field object
String getName()                    Returns the name of the field represented by this
                                    Field object
int getModifiers()                  Returns the modifiers for the field represented by this
                                    Field object encoded as an int
Object get(Object obj)              Returns the value in the specified object of the field 
                                    represented by this Field
------------------------------------------------------------------------------------------------
```

Each `Field` (or `Method`) can have the following modifiers associated: `public static native volatile protected abstract
synchronized strictfp private final`. For example, to determine whether a class has a certain modifier:

In [ ]:
Field nameField = clazz.getDeclaredField("nameField");
Modifier.isStatic(nameField.getModifiers());
Modifier.isVolatile(nameField.getModifiers());

Therefore, to get the instance variables of a class, we can call:

In [ ]:
public Field[] getAllInstanceVariables(Object obj) {
    Class clazz = obj.getClass();
    List<Field> fieldList = new ArrayList<>();
    
    while(clazz != null) {
        Field[] fields = clazz.getDeclaredFields();
        for(Field field: fields){
            if (!Modifier.isStatic(field.getModifiers())) {
                fieldList.add(field);
            }
        }
        clazz = clazz.getSuperclass();
    }
    
    return (Field[]) fieldList.toArray(new Field[fieldList.size()]);
}

**Accessing private members:** For field of type `Field`, `field.setAccessible(true);` disables all runtime access checks on uses of the metaobject referred to by field. This allows reflective access to its value from outside the scope of its visibility.  
The `setAccessible` method works for all of the `get` and `set` methods of `Field` and also for the `invoke` method of `Method`.  
The above method is present in `AccessibleObject` class which is the parent class for both `Field` and `Method`.  
Setting objects as accessible can be disabled in the security manager. If this feature has been disabled, the `setAccessible` methods each throw a `SecurityException`. The default security manager permits the use of setAccessible on members of classes loaded by the same class loader as the caller.  

**Arrays:** Java provides `java.lang.reflect.Array` as a convenience facility for performing reflective operations on all array objects.

```
------------------------------------------------------------------------------------------------
Method                                        Description
------------------------------------------------------------------------------------------------
Object newInstance(Class componentType, int length)
                                       Creates a new array that has the specified component
                                       type and length.
Object newInstance(Class elementType, int[] dimensions)
                                       Creates a new array that has the specified element type
                                       and dimensions.length dimensions.
int getLength( Object array)           Returns the number of components of the specified
                                       array.
Object get(Object array, int index)    Returns the component value at index. Wraps primitives
                                       if necessary.
boolean getBoolean(Object array, int index)
...
                                       If the component type of the specified array is boolean,
                                       the component value at index is returned.
void set(Object array, int index, Object value)
                                       Sets the component at index to the specified value.
                                       Unwraps primitives, if necessary.
void setBoolean(Object array, int index, boolean value)
...
                                       If the component type of the specified array is boolean,
                                       the component at index is set to the specified value.
------------------------------------------------------------------------------------------------
```